*Accompanying code examples of the book "Introduction to Artificial Neural Networks and Deep Learning: A Practical Guide with Applications in Python" by [Sebastian Raschka](https://sebastianraschka.com). All code examples are released under the [MIT license](https://github.com/rasbt/deep-learning-book/blob/master/LICENSE). If you find this content useful, please consider supporting the work by buying a [copy of the book](https://leanpub.com/ann-and-deeplearning).*
  
Other code examples and content are available on [GitHub](https://github.com/rasbt/deep-learning-book). The PDF and ebook versions of the book are available through [Leanpub](https://leanpub.com/ann-and-deeplearning).

In [1]:
%load_ext watermark
%watermark -a 'Sebastian Raschka' -v -p torch

Sebastian Raschka 

CPython 3.6.8
IPython 7.2.0

torch 1.1.0


- Runs on CPU or GPU (if available)

# Model Zoo -- Ordinal Regression CNN -- CORAL

Implementation of a method for ordinal regression, CORAL [1] (COnsistent RAnk Logits) applied to predicting age from face images in the AFAD [2] (Asian Face) dataset using a simple AlexNet [3] convolutional network architecture.

- [1] Cao, Wenzhi, Vahid Mirjalili, and Sebastian Raschka. "[Consistent Rank Logits for Ordinal Regression with Convolutional Neural Networks](https://arxiv.org/abs/1901.07884)." arXiv preprint arXiv:1901.07884 (2019).
- [2] Niu, Zhenxing, Mo Zhou, Le Wang, Xinbo Gao, and Gang Hua. "[Ordinal regression with multiple output cnn for age estimation](https://ieeexplore.ieee.org/document/7780901/)." In Proceedings of the IEEE conference on computer vision and pattern recognition, pp. 4920-4928. 2016.
- [3] Krizhevsky, Alex, Ilya Sutskever, and Geoffrey E. Hinton. "[Imagenet classification with deep convolutional neural networks](http://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networ)." In Advances in neural information processing systems, pp. 1097-1105. 2012.


## Imports

In [2]:
import time
import numpy as np
import pandas as pd
import os

import torch.nn as nn
import torch.nn.functional as F
import torch

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
from PIL import Image


if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True

## Downloading the Dataset

In [3]:
!git clone https://github.com/afad-dataset/tarball.git

fatal: destination path 'tarball' already exists and is not an empty directory.


In [4]:
!cat tarball/AFAD-Full.tar.xz* > AFAD-Full.tar.xz

In [5]:
!tar xf AFAD-Full.tar.xz

In [6]:
!wget https://sebastianraschka.com/datasets/afad/test_set_class0_max25.csv

--2019-06-02 22:33:32--  https://sebastianraschka.com/datasets/afad/test_set_class0_max25.csv
Resolving sebastianraschka.com (sebastianraschka.com)... 67.20.73.179
Connecting to sebastianraschka.com (sebastianraschka.com)|67.20.73.179|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1331840 (1.3M) [text/csv]
Saving to: ‘test_set_class0_max25.csv.9’

test_set_class0_max 100%[===================>]   1.27M  4.49MB/s    in 0.3s    

2019-06-02 22:33:33 (4.49 MB/s) - ‘test_set_class0_max25.csv.9’ saved [1331840/1331840]



In [7]:
!wget https://sebastianraschka.com/datasets/afad/train_set_class0_max25.csv

--2019-06-02 22:33:33--  https://sebastianraschka.com/datasets/afad/train_set_class0_max25.csv
Resolving sebastianraschka.com (sebastianraschka.com)... 67.20.73.179
Connecting to sebastianraschka.com (sebastianraschka.com)|67.20.73.179|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5352951 (5.1M) [text/csv]
Saving to: ‘train_set_class0_max25.csv.9’

train_set_class0_ma 100%[===================>]   5.10M  12.5MB/s    in 0.4s    

2019-06-02 22:33:34 (12.5 MB/s) - ‘train_set_class0_max25.csv.9’ saved [5352951/5352951]



## Settings

In [8]:
##########################
### SETTINGS
##########################

# Device
DEVICE = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")

NUM_WORKERS = 8

NUM_CLASSES = 26
BATCH_SIZE = 256
NUM_EPOCHS = 150
LEARNING_RATE = 0.00005
RANDOM_SEED = 123

TRAIN_CSV_PATH = 'train_set_class0_max25.csv'
TEST_CSV_PATH = 'test_set_class0_max25.csv'
IMAGE_PATH = 'AFAD-Full'

## Dataset Loaders

In [9]:
class AFADDatasetAge(Dataset):
    """Custom Dataset for loading AFAD face images"""

    def __init__(self, csv_path, img_dir, transform=None):

        df = pd.read_csv(csv_path, index_col=0)
        self.img_dir = img_dir
        self.csv_path = csv_path
        self.img_paths = df['path']
        self.y = df['age'].values
        self.transform = transform

    def __getitem__(self, index):
        img = Image.open(os.path.join(self.img_dir,
                                      self.img_paths[index]))

        if self.transform is not None:
            img = self.transform(img)

        label = self.y[index]
        levels = [1]*label + [0]*(NUM_CLASSES - 1 - label)
        levels = torch.tensor(levels, dtype=torch.float32)

        return img, label, levels

    def __len__(self):
        return self.y.shape[0]


custom_transform = transforms.Compose([transforms.Resize((128, 128)),
                                       transforms.RandomCrop((120, 120)),
                                       transforms.ToTensor()])

train_dataset = AFADDatasetAge(csv_path=TRAIN_CSV_PATH,
                               img_dir=IMAGE_PATH,
                               transform=custom_transform)


custom_transform2 = transforms.Compose([transforms.Resize((128, 128)),
                                        transforms.CenterCrop((120, 120)),
                                        transforms.ToTensor()])

test_dataset = AFADDatasetAge(csv_path=TEST_CSV_PATH,
                              img_dir=IMAGE_PATH,
                              transform=custom_transform2)


train_loader = DataLoader(dataset=train_dataset,
                          batch_size=BATCH_SIZE,
                          shuffle=True,
                          num_workers=NUM_WORKERS)

test_loader = DataLoader(dataset=test_dataset,
                         batch_size=BATCH_SIZE,
                         shuffle=False,
                         num_workers=NUM_WORKERS)

## Model

In [10]:
##########################
# MODEL
##########################

class AlexNet(nn.Module):

    def __init__(self, num_classes):
        super(AlexNet, self).__init__()
        self.num_classes = num_classes
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
        )

        self.fc = nn.Linear(4096, 1, bias=False)
        self.linear_1_bias = nn.Parameter(torch.zeros(self.num_classes-1).float())

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), 256 * 6 * 6)
        x = self.classifier(x)

        logits = self.fc(x)
        logits = logits + self.linear_1_bias
        probas = torch.sigmoid(logits)
        return logits, probas

In [11]:
###########################################
# Initialize Cost, Model, and Optimizer
###########################################

def cost_fn(logits, levels):
    val = (-torch.sum((F.logsigmoid(logits)*levels
                      + (F.logsigmoid(logits) - logits)*(1-levels)),
           dim=1))
    return torch.mean(val)


torch.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed(RANDOM_SEED)
model = AlexNet(NUM_CLASSES)

model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

## Training

In [12]:
def compute_mae_and_mse(model, data_loader, device):
    mae, mse, num_examples = 0, 0, 0
    for i, (features, targets, levels) in enumerate(data_loader):

        features = features.to(device)
        targets = targets.to(device)

        logits, probas = model(features)
        predict_levels = probas > 0.5
        predicted_labels = torch.sum(predict_levels, dim=1)
        num_examples += targets.size(0)
        mae += torch.sum(torch.abs(predicted_labels - targets))
        mse += torch.sum((predicted_labels - targets)**2)
    mae = mae.float() / num_examples
    mse = mse.float() / num_examples
    return mae, mse


start_time = time.time()
for epoch in range(NUM_EPOCHS):

    model.train()
    for batch_idx, (features, targets, levels) in enumerate(train_loader):

        features = features.to(DEVICE)
        targets = targets
        targets = targets.to(DEVICE)
        levels = levels.to(DEVICE)

        # FORWARD AND BACK PROP
        logits, probas = model(features)
        cost = cost_fn(logits, levels)
        optimizer.zero_grad()

        cost.backward()

        # UPDATE MODEL PARAMETERS
        optimizer.step()

        # LOGGING
        if not batch_idx % 150:
            s = ('Epoch: %03d/%03d | Batch %04d/%04d | Cost: %.4f'
                 % (epoch+1, NUM_EPOCHS, batch_idx,
                     len(train_dataset)//BATCH_SIZE, cost))
            print(s)

    s = 'Time elapsed: %.2f min' % ((time.time() - start_time)/60)
    print(s)

Epoch: 001/150 | Batch 0000/0514 | Cost: 17.3278
Epoch: 001/150 | Batch 0150/0514 | Cost: 16.5859
Epoch: 001/150 | Batch 0300/0514 | Cost: 16.4164
Epoch: 001/150 | Batch 0450/0514 | Cost: 16.0580
Time elapsed: 0.59 min
Epoch: 002/150 | Batch 0000/0514 | Cost: 16.2263
Epoch: 002/150 | Batch 0150/0514 | Cost: 16.0722
Epoch: 002/150 | Batch 0300/0514 | Cost: 15.9567
Epoch: 002/150 | Batch 0450/0514 | Cost: 16.0964
Time elapsed: 1.18 min
Epoch: 003/150 | Batch 0000/0514 | Cost: 15.8509
Epoch: 003/150 | Batch 0150/0514 | Cost: 15.7487
Epoch: 003/150 | Batch 0300/0514 | Cost: 15.6847
Epoch: 003/150 | Batch 0450/0514 | Cost: 15.3686
Time elapsed: 1.78 min
Epoch: 004/150 | Batch 0000/0514 | Cost: 15.2173
Epoch: 004/150 | Batch 0150/0514 | Cost: 15.3359
Epoch: 004/150 | Batch 0300/0514 | Cost: 15.2410
Epoch: 004/150 | Batch 0450/0514 | Cost: 15.1033
Time elapsed: 2.39 min
Epoch: 005/150 | Batch 0000/0514 | Cost: 15.2330
Epoch: 005/150 | Batch 0150/0514 | Cost: 15.1501
Epoch: 005/150 | Batch 030

## Evaluation

In [13]:
model.eval()
with torch.set_grad_enabled(False):  # save memory during inference

    train_mae, train_mse = compute_mae_and_mse(model, train_loader,
                                               device=DEVICE)
    test_mae, test_mse = compute_mae_and_mse(model, test_loader,
                                             device=DEVICE)

    s = 'MAE/RMSE: | Train: %.2f/%.2f | Test: %.2f/%.2f' % (
        train_mae, torch.sqrt(train_mse), test_mae, torch.sqrt(test_mse))
    print(s)

s = 'Total Training Time: %.2f min' % ((time.time() - start_time)/60)
print(s)

MAE/RMSE: | Train: 1.19/1.64 | Test: 3.78/5.14
Total Training Time: 112.72 min


In [14]:
%watermark -iv

numpy       1.15.4
pandas      0.23.4
torch       1.1.0
PIL.Image   5.3.0

